In [1]:
import gdspy
import numpy as np
import random
from datetime import datetime

def create_e_letter(cell, position, e_size, gap, orientation):
    """
    在指定位置绘制一个E字形状，具有给定的大小和方向。
    """
    w = e_size / 4  # 定义E字线宽
    pts_vertical = [(position[0], position[1]), (position[0], position[1] + e_size)]
    pts_horizontal = [
        (position[0], position[1] + e_size * 0.8),
        (position[0] + e_size - gap, position[1] + e_size * 0.8),
        (position[0], position[1] + e_size * 0.5),
        (position[0] + e_size - gap, position[1] + e_size * 0.5),
        (position[0], position[1] + e_size * 0.2),
        (position[0] + e_size - gap, position[1] + e_size * 0.2)
    ]
    cell.add(gdspy.Polygon(pts_vertical, layer=1).rotate(orientation, position))
    for i in range(0, len(pts_horizontal), 2):
        cell.add(gdspy.Polygon([pts_horizontal[i], pts_horizontal[i+1], (pts_horizontal[i+1][0], pts_horizontal[i+1][1] - w), (pts_horizontal[i][0], pts_horizontal[i][1] - w)], layer=1).rotate(orientation, position))

# 初始化gdspy
gdspy.current_library = gdspy.GdsLibrary()
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
cell_name = f'DOT_ARRAY_WITH_FRAME_{current_time}'
cell = gdspy.Cell(cell_name)

# 定义参数
start_gap = 30  # 起始间隔30um
end_gap = 1    # 终止间隔1um
gap_decrement = 1  # 间隔递减量
e_size = 100  # E字的大小
e_spacing = 2 * e_size  # E字之间的间隔

x_position = 0
y_position = 0

# 绘制视力表
for gap in range(start_gap, end_gap - 1, -gap_decrement):
    x_position = 0  # 每次开始新的一行时重置x坐标
    for _ in range(5):  # 假设每行绘制5个E
        orientation = random.choice([0, np.pi/2, np.pi, 3*np.pi/2])  # 随机选择方向
        create_e_letter(cell, (x_position, y_position), e_size, gap, orientation)
        x_position += e_spacing  # 移动到下一个E的位置
    # 在行的末尾添加间隔标注
    text = gdspy.Text(f'{gap}um', 12, (x_position, y_position))
    cell.add(text)
    y_position -= e_size * 1.5  # 移动到下一行

# 保存和显示结果
gdspy.write_gds('vision_test.gds')
print("GDS文件已生成:")


GDS文件已生成:


In [2]:
import gdspy
import numpy as np

def draw_e(cell, origin, e_height, line_width, gap, orientation=0):
    """
    在指定的单元格中绘制一个E字形状。

    参数:
    - cell: gdspy.Cell对象，绘制E字的单元格。
    - origin: (x, y)元组，E字形状的左下角坐标。
    - e_height: E字的高度。
    - line_width: E字的线宽。
    - gap: E字内两横线之间的距离。
    - orientation: E字的开口朝向，以弧度为单位（默认为0，即朝右）。
    """
    x, y = origin
    # 创建竖线
    vertical_line = gdspy.Rectangle((x, y), (x + line_width, y + e_height))
    # 创建上横线
    top_horizontal_line = gdspy.Rectangle((x, y + e_height - line_width), (x + 3*line_width + 2*gap, y + e_height))
    # 创建中横线
    middle_horizontal_line = gdspy.Rectangle((x, y + (e_height/2) - (line_width/2)), (x + 2*line_width + gap, y + (e_height/2) + (line_width/2)))
    # 创建下横线
    bottom_horizontal_line = gdspy.Rectangle((x, y), (x + 3*line_width + 2*gap, y + line_width))

    # 将形状添加到单元格
    for shape in [vertical_line, top_horizontal_line, middle_horizontal_line, bottom_horizontal_line]:
        # 如果需要，旋转形状
        if orientation != 0:
            shape.rotate(orientation, center=origin)
        cell.add(shape)


In [3]:
# 初始化gdspy库
gdspy.current_library = gdspy.GdsLibrary()
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
cell_name = f'DOT_ARRAY_WITH_FRAME_{current_time}'
# 创建一个新的单元格
cell = gdspy.Cell(cell_name)

# 绘制E字形状
e_height = 100  # E的高度
line_width = 10  # 线宽
gap = 20  # 两横线之间的距离
origin = (0, 0)  # E字的起点
orientation = np.pi / 2  # E的开口朝向，此处示例为向上

# 调用函数绘制E
draw_e(cell, origin, e_height, line_width, gap, orientation)

# 保存GDS文件
gdspy.write_gds('e_letter_example.gds')

# 如果你有适合的查看器，可以查看保存的GDS文件
print("GDS文件已生成，文件名为：e_letter_example.gds")


GDS文件已生成，文件名为：e_letter_example.gds


In [5]:
import gdspy
import numpy as np
from datetime import datetime
def draw_e_with_time_named_cell(line_width, gap, origin, orientation=0):
    """
    在一个以当前时间命名的单元格中绘制E字形状，保证三条横线长度相等。

    参数:
    - line_width: E字的线宽。
    - gap: E字内两横线之间的距离。
    - origin: (x, y)元组，E字形状的左下角坐标。
    - orientation: E字的开口朝向，以弧度为单位（默认为0，即朝右）。
    """
    # 计算E的高度
    e_height = 3 * line_width + 2 * gap
    # 确定横线长度
    e_length = 3 * line_width + 2 * gap

    x, y = origin
    # 创建竖线
    vertical_line = gdspy.Rectangle((x, y), (x + line_width, y + e_height)).rotate(orientation, center=origin)
    # 创建三条横线
    horizontal_lines = [
        gdspy.Rectangle((x, y + i * (line_width + gap)), (x + e_length, y + line_width + i * (line_width + gap))).rotate(orientation, center=origin) for i in range(3)
    ]

    # 生成单元格名称
    cell_name = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    cell = gdspy.Cell(cell_name)

    # 将形状添加到单元格
    cell.add(vertical_line)
    for line in horizontal_lines:
        cell.add(line)

    return cell

# 初始化gdspy库
# 使用示例
line_width = 10  # 线宽
gap = 15  # 两横线之间的距离
origin = (0, 0)  # E字的起点
orientation = np.pi / 2  # E的开口朝向，示例为向上

gdspy.current_library = gdspy.GdsLibrary()

# 使用示例，其他参数保持不变
cell = draw_e_with_time_named_cell(line_width, gap, origin, orientation)

# 保存GDS文件
gdspy.write_gds('e_letter_with_time_named_cell_corrected.gds')

print("GDS文件已生成，包含一个名为'{}'的单元格，文件名为：e_letter_with_time_named_cell_corrected.gds".format(cell.name))



GDS文件已生成，包含一个名为'20240315_151211_917111'的单元格，文件名为：e_letter_with_time_named_cell_corrected.gds


In [7]:
def draw_e_with_squares_and_time_named_cell(line_width, gap, origin, orientation=0):
    """
    在一个以当前时间命名的单元格中绘制E字形状及其四周的正方形，保证三条横线长度相等。

    参数:
    - line_width: E字的线宽。
    - gap: E字内两横线之间的距离及正方形与E字之间的间隔。
    - origin: (x, y)元组，E字形状的左下角坐标。
    - orientation: E字的开口朝向，以弧度为单位（默认为0，即朝右）。
    """
    # 计算E的高度和横线长度
    e_height = 3 * line_width + 2 * gap
    e_length = 3 * line_width + 2 * gap
    square_size = gap / 4

    x, y = origin
    # 创建竖线
    vertical_line = gdspy.Rectangle((x, y), (x + line_width, y + e_height))
    # 创建三条横线
    horizontal_lines = [
        gdspy.Rectangle((x, y + i * (line_width + gap)), (x + e_length, y + line_width + i * (line_width + gap))) for i in range(3)
    ]

    # 合并E字形状
    e_shape = gdspy.boolean(vertical_line, horizontal_lines, 'or')

    # 计算正方形的位置并创建
    squares = [
        gdspy.Rectangle((x - gap, y + e_height + gap), (x - gap + square_size, y + e_height + gap + square_size)),
        gdspy.Rectangle((x + e_length + gap - square_size, y + e_height + gap), (x + e_length + gap, y + e_height + gap + square_size)),
        gdspy.Rectangle((x - gap, y - gap - square_size), (x - gap + square_size, y - gap)),
        gdspy.Rectangle((x + e_length + gap - square_size, y - gap - square_size), (x + e_length + gap, y - gap))
    ]

    # 合并E字形状和正方形
    final_shape = gdspy.boolean(e_shape, squares, 'or')

    # 如果需要，旋转整体形状
    if orientation != 0:
        final_shape.rotate(orientation, center=origin)

    # 生成单元格名称
    cell_name = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    cell = gdspy.Cell(cell_name)

    # 将合并后的形状添加到单元格
    cell.add(final_shape)

    return cell
# 初始化gdspy库
# 使用示例
line_width = 10  # 线宽
gap = 15  # 两横线之间的距离
origin = (0, 0)  # E字的起点
orientation = np.pi / 2  # E的开口朝向，示例为向上

gdspy.current_library = gdspy.GdsLibrary()

# 使用示例，其他参数保持不变
cell = draw_e_with_squares_and_time_named_cell(line_width, gap, origin, orientation)

# 保存GDS文件
gdspy.write_gds('e_letter_with_time_named_cell_corrected.gds')

print("GDS文件已生成，包含一个名为'{}'的单元格，文件名为：e_letter_with_time_named_cell_corrected.gds".format(cell.name))

GDS文件已生成，包含一个名为'20240315_152637_118463'的单元格，文件名为：e_letter_with_time_named_cell_corrected.gds


In [11]:
def draw_e_with_adjusted_squares_and_time_named_cell(line_width, gap, origin, orientation=0):
    """
    绘制E字形状及其周围，根据新距离条件放置的四个小正方形。

    参数:
    - line_width: E字的线宽。
    - gap: E字内两横线之间的距离。
    - origin: (x, y)元组，E字形状的左下角坐标。
    - orientation: E字的开口朝向，以弧度为单位（默认为0，即朝右）。
    """
    # 计算E的高度和横线长度
    e_height = 3 * line_width + 2 * gap
    e_length = 3 * line_width + 2 * gap
    square_size = gap / 4
    x, y = origin

    # 创建竖线和三条横线，合并为E字形状
    vertical_line = gdspy.Rectangle((x, y), (x + line_width, y + e_height))
    horizontal_lines = [
        gdspy.Rectangle((x, y + i * (line_width + gap)), (x + e_length, y + line_width + i * (line_width + gap))) for i in range(3)
    ]
    e_shape = gdspy.boolean(vertical_line, horizontal_lines, 'or')

    # 根据新的间隔条件计算小正方形的位置
    offset = 3 * line_width / 2 + 2 * gap
    squares_positions = [
        (x + e_length / 2 - square_size / 2, y + offset - square_size / 2),  # 上方
        (x + offset - square_size / 2, y + e_height / 2 - square_size / 2),  # 右侧
        (x + e_length / 2 - square_size / 2, y - offset),  # 下方
        (x - offset, y + e_height / 2 - square_size / 2)  # 左侧
    ]
    squares = [gdspy.Rectangle((pos[0], pos[1]), (pos[0] + square_size, pos[1] + square_size)) for pos in squares_positions]

    # 合并E字形状和正方形
    final_shape = gdspy.boolean(e_shape, squares, 'or')

    # 如果需要，旋转整体形状
    if orientation != 0:
        final_shape.rotate(orientation, center=(x + e_length / 2, y + e_height / 2))

    # 生成单元格名称
    cell_name = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    cell = gdspy.Cell(cell_name)

    # 将合并后的形状添加到单元格
    cell.add(final_shape)

    return cell

# 初始化gdspy库
# 使用示例
line_width = 10  # 线宽
gap = 10  # 两横线之间的距离
origin = (0, 0)  # E字的起点
orientation = np.pi / 2  # E的开口朝向，示例为向上

gdspy.current_library = gdspy.GdsLibrary()

# 使用示例，其他参数保持不变
cell = draw_e_with_centered_squares_and_time_named_cell(line_width, gap, origin, orientation)

# 保存GDS文件
gdspy.write_gds('e_letter_with_time_named_cell_corrected.gds')

print("GDS文件已生成，包含一个名为'{}'的单元格，文件名为：e_letter_with_time_named_cell_corrected.gds".format(cell.name))

GDS文件已生成，包含一个名为'20240315_153535_276802'的单元格，文件名为：e_letter_with_time_named_cell_corrected.gds


In [38]:
import uuid
def draw_e_with_adjusted_squares(line_width, gap, origin, orientation=0, cell_name="E_Cell"):
    """
    绘制E字形状及其周围，根据新距离条件放置的四个小正方形，并允许外部指定单元格名称。

    参数:
    - line_width: E字的线宽。
    - gap: E字内两横线之间的距离。
    - origin: (x, y)元组，E字形状的左下角坐标。
    - orientation: E字的开口朝向，以弧度为单位（默认为0，即朝右）。
    - cell_name: 单元格的名称，确保每次调用都是唯一的。
    """
    e_height = 3 * line_width + 2 * gap
    e_length = 3 * line_width + 2 * gap
    square_size = max(gap / 4, 1)
    x, y = origin

    # 创建单元格
    cell_name = "E_Cell_" + str(uuid.uuid4())
    cell = gdspy.Cell(cell_name)

    # 创建竖线和三条横线，合并为E字形状
    vertical_line = gdspy.Rectangle((x, y), (x + line_width, y + e_height))
    horizontal_lines = [
        gdspy.Rectangle((x, y + i * (line_width + gap)), (x + e_length, y + line_width + i * (line_width + gap))) for i in range(3)
    ]
    e_shape = gdspy.boolean(vertical_line, horizontal_lines, 'or')

    # 计算小正方形的位置
    squares_positions = [
        (x + e_length / 2 - square_size / 2, y + e_height + gap - square_size / 2),
        (x + e_length + gap - square_size / 2, y + e_height / 2 - square_size / 2),
        (x + e_length / 2 - square_size / 2, y - gap - square_size / 2),
        (x - gap - square_size / 2, y + e_height / 2 - square_size / 2)
    ]
    squares = [gdspy.Rectangle((pos[0], pos[1]), (pos[0] + square_size, pos[1] + square_size)) for pos in squares_positions]

    # 合并E字形状和正方形
    final_shape = gdspy.boolean(e_shape, squares, 'or')

    # 如果需要，旋转整体形状
    if orientation != 0:
        final_shape.rotate(orientation, center=(x + e_length / 2, y + e_height / 2))

    # 将合并后的形状添加到单元格
    cell.add(final_shape)

    return cell


In [39]:
from datetime import datetime
import random
import gdspy

# 初始化gdspy库
gdspy.current_library = gdspy.GdsLibrary()

width_gap_values = [30, 25, 20, 15, 10, 8, 6, 4, 3, 2, 1]
origin_x, origin_y = 0, 0  # 起始原点坐标
main_cell_name = "VISION_TEST_" + datetime.now().strftime("%Y%m%d_%H%M%S")
main_cell = gdspy.Cell(main_cell_name)

# 自定义每一行E的个数
e_counts_per_row = [1, 2, 3, 4, 6, 10, 20, 40, 60, 80, 100]  # 示例，根据需要调整

# 用于记录上一行E字的大小，初始化为列表中的第一个值
previous_e_size = 3 * width_gap_values[0] + 2 * width_gap_values[0]

for i, (value, num_es) in enumerate(zip(width_gap_values, e_counts_per_row)):
    line_width = gap = value
    # 计算单个E的总宽度（包括周围的小正方形）
    e_size = 3 * line_width + 2 * gap
    # E之间的间隔空出两个E的大小
    spacing = 4 * e_size

    # 计算行宽，用于居中对齐
    row_width = num_es * e_size + (num_es - 1) * spacing
    # 居中对齐的起始x_position
    x_position = origin_x + (width_gap_values[0] * 3 + 2 * width_gap_values[0] - row_width) / 2  # 基于最大E宽度进行居中

    # 更新origin_y，使得行间距等于上一行E字的大小
    if i > 0:  # 从第二行开始更新
        origin_y -= previous_e_size + 2*e_size  # 行间距为上一行E的大小

    # 遍历每一行中的每个E
    for j in range(num_es):
        orientation = random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        e_origin = (x_position + j * (e_size + spacing), origin_y)
        e_cell_name = f"E_{i}_{j}_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
        e_cell = draw_e_with_adjusted_squares(line_width, gap, e_origin, orientation, e_cell_name)
        main_cell.add(gdspy.CellReference(e_cell, origin=(0, 0)))

    # 更新previous_e_size为当前行E的大小，供下一行使用
    previous_e_size = e_size

# 保存GDS文件
filename = "vision_test_with_squares.gds"
gdspy.write_gds(filename)
print(f"视力表已生成并保存为：{filename}")


视力表已生成并保存为：vision_test_with_squares.gds
